Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [4]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from tqdm import tqdm 

In [5]:
pickle_file = 'nlp/NLP_assignments/assignment-09/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [6]:
image_size = 28
num_labels = 10
num_channels = 1 # gray scale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [7]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) # patch_size -> kernel_size, num_channels-> last dim -> this dim
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))  # fully connected layer height = 16/2 16/2  所以是16//4 
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [-1, shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



## same padding 
最低限度填充

new = 向上取整w/s

In [ ]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step % 50 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.753566
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.113889
Minibatch accuracy: 31.2%
Validation accuracy: 33.1%
Minibatch loss at step 100: 1.510559
Minibatch accuracy: 50.0%
Validation accuracy: 58.2%
Minibatch loss at step 150: 0.417214
Minibatch accuracy: 93.8%
Validation accuracy: 74.0%
Minibatch loss at step 200: 0.666050
Minibatch accuracy: 75.0%
Validation accuracy: 77.6%
Minibatch loss at step 250: 0.610765
Minibatch accuracy: 87.5%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.455869
Minibatch accuracy: 81.2%
Validation accuracy: 78.7%
Minibatch loss at step 350: 0.489266
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 400: 0.767434
Minibatch accuracy: 68.8%
Validation accuracy: 78.2%
Minibatch loss at step 450: 0.810616
Minibatch accuracy: 68.8%
Validation accuracy: 80.5%
Minibatch loss at step 500: 0.697201
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [9]:
class Model:
    def __init__(self, graph=None):
        if not graph:
            self.graph = tf.Graph()
        else:
            self.graph = graph
    
    def build_graph(self, image_size, channel, num_labels, kernel_size, depth, flat_units): # keep_prob should in the train process 
        with self.graph.as_default():
            self.feature = tf.placeholder(tf.float32, shape=[None, image_size, image_size, channel])
            self.labels = tf.placeholder(tf.float32, shape = [None, num_labels])
            self.keep_prob = tf.placeholder(tf.float32, shape = ())
            
            filter1= tf.get_variable("f1",shape=[kernel_size,kernel_size, channel, depth],initializer=tf.initializers.glorot_normal())
            biases1 = tf.get_variable("b1",shape = [depth], initializer=tf.initializers.glorot_normal())
            layer1 = tf.nn.relu(tf.nn.conv2d(self.feature,filter1,[1,2,2,1],"SAME")+biases1)
            dropout1 = tf.nn.dropout(layer1,keep_prob=self.keep_prob)
            
            filter2 = tf.get_variable("f2", shape=[kernel_size,kernel_size,depth, depth], initializer=tf.initializers.glorot_normal())
            biases2 = tf.get_variable("b2", shape=[depth])
            layer2 = tf.nn.relu(tf.nn.conv2d(dropout1,filter2,strides=[1,2,2,1],padding="SAME")+biases2)
                        
            drop_shape = layer2.shape
            reshaped = tf.reshape(layer2, shape=[-1,drop_shape[1]*drop_shape[2]*drop_shape[3]])
            dropout2 = tf.nn.dropout(reshaped,keep_prob=self.keep_prob)
            transformed_size = image_size // 4
            
            weight3 = tf.get_variable("w3", shape =[transformed_size*transformed_size*depth, flat_units],initializer=tf.initializers.glorot_normal())
            biases3 = tf.get_variable("b3", shape=[flat_units])
            
            fully_connected= tf.nn.relu(tf.matmul(dropout2,weight3)+biases3)
            
            weight4 = tf.get_variable("w4", shape = [flat_units, num_labels],initializer=tf.initializers.glorot_normal())
            biases4 = tf.get_variable("b4", shape = [num_labels],initializer=tf.initializers.glorot_normal())
            
            output_logits = tf.matmul(fully_connected,weight4)+biases4
            
            
            self.loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits=output_logits,labels=self.labels)
            )
            self.optimizer = tf.train.AdagradOptimizer(0.005).minimize(self.loss)
            
            self.prediction = tf.nn.softmax(output_logits)
            
            self.accuracy = np.mean(tf.argmax(self.prediction,axis=1) == tf.argmax(self.labels))
        
    def make_dataset(self, x, y , batch_size=50):
        origin_dict = (x,y)
        dataset = tf.data.Dataset.from_tensor_slices(origin_dict)
        dataset = dataset.shuffle()  # buffer_size 每次从多少个数据中抽样
        dataset = dataset.batch(batch_size)
        dataset = dataset.repeat()
        dataset_iter = dataset.make_one_shot_iterator()
        next_iter = dataset_iter.get_next()
        return next_iter
    
    def train(self,data_dict, epoch_num, dropout_rate=0.7):
        
        train_iter = self.make_dataset(data_dict["train_x"],data_dict["train_y"])
        dev_iter = self.make_dataset(data_dict["dev_x"],data_dict["dev_y"])
        test_iter = self.make_dataset(data_dict["test_x"],data_dict["test_y"])
        
        
        with tf.Session(graph=self.graph) as sess:
            tf.global_variables_initializer().run()
            for i in tqdm(epoch_num):
                train_tuple = sess.run(train_iter)
                train_dict = {self.feature:train_tuple[0],
                              self.labels:train_tuple[1],
                              self.keep_prob:dropout_rate}
                _, loss = sess.run([self.optimizer,self.loss],
                                   feed_dict=train_dict)
                if i%100 ==0:
                    dev_tuple= sess.run(dev_iter)
                    dev_dict = {self.feature:dev_tuple[0],
                              self.labels:dev_tuple[1],
                              self.keep_prob:dropout_rate}
                    dev_accuracy = sess.run(self.accuracy,feed_dict=dev_dict)
                    train_accuracy = sess.run(self.accuracy,feed_dict= train_dict)
                    print("accuracy on train dataset %s" %train_accuracy)
                    print("accuracy on dev dataset %s " %dev_accuracy)
                test_tuple = sess.run(test_iter)
                train_dict = {self.feature:test_tuple[0],
                              self.labels:test_tuple[1],
                              self.keep_prob:dropout_rate}
                test_accuracy = sess.run(self.accuracy,feed_dict=train_dict)
                print("accuracy on test dataset %s" %test_accuracy)                  
            
    
    def evaluate(self, X, Y):
        pass  

In [ ]:
new_model = Model()
new_model.build_graph(image_size,num_channels,num_labels,patch_size,depth,num_hidden)
print("build graph successfully")
data_dict = {
    "train_x":train_dataset,
    "train_y":train_labels,
    "dev_x":valid_dataset,
    "dev_y":valid_labels,
    "test_x":test_dataset,
    "test_y":test_labels
}
new_model.train(data_dict,200)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---